学号:2010349

姓名:孟笑朵

实验笔记全部已注释的形式放到代码块中了

# HW1 布尔检索

本次作业使用斯坦福大学[CS 276 / LING 286: Information Retrieval and Web Search](https://web.stanford.edu/class/cs276/)课程的代码框架来实现。具体来说主要包含的内容有：
1. [索引构建 (40%)](#索引构建与检索-(40%)) 使用BSBI方法模拟在内存不足的情况下的索引构建方式，并应用于布尔查询
2. [索引压缩 (30%)](#索引压缩-(30%)) 使用可变长编码对构建的索引进行压缩
3. [布尔检索 (10%)](#布尔联合检索-(10%)) 对空格分隔的单词查询进行联合（与）布尔检索
3. [实验报告 (10%)](#Report-(25%)) 描述你的代码并回答一些问题
4. [额外的编码方式 (10%)](#额外的编码方式-(10%)) 鼓励使用额外的编码方式对索引进行压缩 (例如, gamma-encoding)

In [356]:
# You can add additional imports here
import sys
import pickle as pkl
import array
import os
import timeit 
import contextlib
import numpy as np

# 数据集

实验使用的文本数据是stanford.edu域下的网页内容，可从http://web.stanford.edu/class/cs276/pa/pa1-data.zip 下载。以下代码将大约170MB的文本数据下载到当前目录下，

In [357]:
# import urllib.request
# import zipfile

# # data_url = 'http://web.stanford.edu/class/cs276/pa/pa1-data.zip'
# # data_dir = 'pa1-data'
# # urllib.request.urlretrieve(data_url, data_dir+'.zip')
# zip_ref = zipfile.ZipFile(data_dir+'.zip', 'r')
# zip_ref.extractall()
# zip_ref.close()

之后构建的索引会被存储到`output_dir`，`tmp`会存储测试数据（toy-data）所生成的一些临时文件

In [358]:
try: 
    os.mkdir('output_dir')
except FileExistsError:
    pass
try: 
    os.mkdir('tmp')
except FileExistsError:
    pass
try: 
    os.mkdir('toy_output_dir')
except FileExistsError:
    pass

在数据目录下有10个子目录（命名0-9）

In [359]:
sorted(os.listdir('pa1-data'))

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

每一个子目录下的文件都包含一个独立网页的内容。可以认为在同一子目录下没有同名文件，即每个文件的绝对路径不会相同。

In [360]:
sorted(os.listdir('pa1-data/0'))[:10]

['3dradiology.stanford.edu_',
 '3dradiology.stanford.edu_patient_care_Case%2520studies_AVM.html',
 '3dradiology.stanford.edu_patient_care_case_studies.html',
 '5-sure.stanford.edu_',
 '50years.stanford.edu_',
 'a3cservices.stanford.edu_awards_nominate_',
 'a3cservices.stanford.edu_facilities_',
 'a3cservices.stanford.edu_lead_',
 'aa.stanford.edu_',
 'aa.stanford.edu_about_aviation.php']

所有的网页内容已经经过处理，仅包含由空格分隔开的单词，不再需要进行额外的标准化工作。

In [361]:
with open('pa1-data/0/3dradiology.stanford.edu_', 'r') as f:
    print(f.read())

3d radiology lab stanford university school of medicine stanford school of medicine 3d and quantitative imaging in the department of radiology search this site only stanford medical sites ways to give find a person alumni lane library ways to give find a person about us mission to develop and apply innovative techniques for efficient quantitative analysis and display of medical imaging data through interdisciplinary collaboration goals education to train physicians and technologists locally and worldwide in the latest developments in 3d and quantitative imaging research to develop new approaches to the exploration analysis and quantitative assesment of diagnostic images that result in a new and or more cost effective diagnostic approaches and b new techniques for the design and planning and monitoring of therapy patient care to deliver valid clinically relevant visualization and analysis of medical imaging data to the stanford community locations richard m lucas magnetic resonance imag

作业目录下有一个小的数据集文件夹`toy-data`。在使用完整网页数据集之前，我们会用它来测试我们的代码是否正确。

In [362]:
toy_dir = 'toy-data'

# 索引构建与检索 (40%)

作业的第一部分是使用**blocked sort-based indexing (BSBI)** 算法来构建倒排索引并实现布尔检索。关于BSBI算法可以参考老师课件或者斯坦福教材[Section 4.2](http://nlp.stanford.edu/IR-book/pdf/04const.pdf)。以下摘自教材内容

> To construct an index, we first make a pass through the collection assembling all term-docID pairs. We then sort the pairs with the term as the dominant key and docID as the secondary key. Finally, we organize the docIDs for each term into a postings list and compute statistics like term and document frequency. For small collections, all this can be done in memory. 

对于无法在内存一次性处理的较大数据集，将会使用到二级存储（如：磁盘）。

## IdMap

再次引用教材 Section 4.2:

> To make index construction more efficient, we represent terms as termIDs (instead of strings), where each termID is a unique serial number. We can build the mapping from terms to termIDs on the fly while we are processing the collection. Similarly, we also represent documents as docIDs (instead of strings).

我们首先定义一个辅助类`IdMap`，用于将字符串和数字ID进行相互映射，以满足我们在term和termID、doc和docID间转换的需求。

实现以下代码中的`_get_str` 和 `_get_id`函数，IdMap类的唯一接口是`__getitem__`，它是一个特殊函数，重写了下标运算`[]`,根据下标运算键的类型得到正确的映射值（如果不存在需要添加）。（特殊函数可参考[官方文档](https://docs.python.org/3.7/reference/datamodel.html#special-method-names)）
<br>
<br>
我们会用到字典来将字符串转换为数字，用列表来将数字转换为字符串。

### 辅助理解

构建词典

In [365]:
class IdMap:
    """Helper class to store a mapping from strings to ids."""
    # 1. 确定字符串和id的存储方式: {"字符串": id} 字典可以实现依据字符来取对应的id值; 
    # 列表的下标本身就代表对应的映射id, 只需要取列表对应下标的字符串即可取得 id值对应的字符串
    # 2. 如果不存在对应的字符串和id值的映射关系, 需要手动添加对应的字符串和id值
    def __init__(self):
        self.str_to_id = {}
        self.id_to_str = []
        
    def __len__(self):
        """Return number of terms stored in the IdMap"""
        return len(self.id_to_str)
        
    def _get_str(self, i):
        """Returns the string corresponding to a given id (`i`)."""
        ### Begin your code
        # 1. 进行合法性检查
        if i >= self.__len__():
            raise IndexError
        else:
            # 2. 返回结果
            return self.id_to_str[i]
        ### End your code
        
    def _get_id(self, s):
        """Returns the id corresponding to a string (`s`). 
        If `s` is not in the IdMap yet, then assigns a new id and returns the new id.
        """
        ### Begin your code
        # 1. 判断字符是否存在
        if s in self.str_to_id:
            return self.str_to_id[s]
        else:
            # 2.如果不存在, 需要添加字符串并且为字符串设置对应的id值
            self.id_to_str.append(s)
            self.str_to_id[s]=len(self.id_to_str)-1
            return self.str_to_id[s]

        ### End your code
            
    def __getitem__(self, key):
        """If `key` is a integer, use _get_str; 
           If `key` is a string, use _get_id;"""
        if type(key) is int:
            return self._get_str(key)
        elif type(key) is str:
            return self._get_id(key)
        else:
            raise TypeError

确保代码能通过以下简单测试样例

In [366]:
testIdMap = IdMap()
assert testIdMap['a'] == 0, "Unable to add a new string to the IdMap"
assert testIdMap['bcd'] == 1, "Unable to add a new string to the IdMap"
assert testIdMap['a'] == 0, "Unable to retrieve the id of an existing string"
assert testIdMap[1] == 'bcd', "Unable to retrive the string corresponding to a\
                                given id"
try:
    testIdMap[2]
except IndexError as e:
    assert True, "Doesn't throw an IndexError for out of range numeric ids"
assert len(testIdMap) == 2

之后会需要你自己来写测试样例来确保你的程序正常运行

## 将倒排列表编码成字节数组

为了高效地从磁盘读写倒排列表（文档ID），我们将其存储为字节数组的形式。代码提供了`UncompressedPostings`类来用静态函数实现对倒排列表的编码和解码。在之后的任务中你需要使用该接口实现索引压缩版本（可变长编码）。

参考:
1. https://docs.python.org/3/library/array.html
2. https://pymotw.com/3/array/#module-array

In [367]:
class UncompressedPostings:
    
    @staticmethod
    def encode(postings_list):
        """Encodes postings_list into a stream of bytes
        
        Parameters
        ----------
        postings_list: List[int]
            List of docIDs (postings)
            
        Returns
        -------
        bytes
            bytearray representing integers in the postings_list
        """
        return array.array('L', postings_list).tobytes()
        
    @staticmethod
    def decode(encoded_postings_list):
        """Decodes postings_list from a stream of bytes
        
        Parameters
        ----------
        encoded_postings_list: bytes
            bytearray representing encoded postings list as output by encode 
            function
            
        Returns
        -------
        List[int]
            Decoded list of docIDs from encoded_postings_list
        """
        
        decoded_postings_list = array.array('L')
        decoded_postings_list.frombytes(encoded_postings_list)
        return decoded_postings_list.tolist()

运行以下代码查看其工作方式

In [368]:
x = UncompressedPostings.encode([1,2,3])
print(x)
print(UncompressedPostings.decode(x))

b'\x01\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00'
[1, 2, 3]


## 磁盘上的倒排索引

> With main memory insufficient, we need to use an external sorting algorithm, that is, one that uses disk. For acceptable speed, the central requirement of such an algorithm is that it minimize the number of random disk seeks during sorting - sequential disk reads are far faster than seeks. 

在这一部分我们提供了一个基类`InvertedIndex`，之后会在此基础上构建它的子类`InvertedIndexWriter`, `InvertedIndexIterator` 和 `InvertedIndexMapper`。在Python中我们常用`cPickle`进行序列化，但是它并不支持部分读和部分写，无法满足BSBI算法的需要，所以我们需要定义自己的存储方式。

### 辅助理解

本类是实现读或者写倒排索引到磁盘中的基类，`InvertedIndexWriter`, `InvertedIndexIterator` 和 `InvertedIndexMapper`中每次打开就可以直接读取postings_dict和terms，其中postings_dict是以三元组的形式存在的，这两部分都是以文章和词项的列表的形式存在的

In [369]:
class InvertedIndex:
    """A class that implements efficient reads and writes of an inverted index 
    to disk
    实现读或者写倒排索引到磁盘中的基类
    
    Attributes
    ----------
    postings_dict: Dictionary mapping: termID->(start_position_in_index_file, 
                                                number_of_postings_in_list,
                                               length_in_bytes_of_postings_list)
        This is a dictionary that maps from termIDs to a 3-tuple of metadata 
        that is helpful in reading and writing the postings in the index file 
        to/from disk. This mapping is supposed to be kept in memory. 
        start_position_in_index_file is the position (in bytes) of the postings 
        list in the index file 
        number_of_postings_in_list is the number of postings (docIDs) in the 
        postings list
        length_in_bytes_of_postings_list is the length of the byte 
        encoding of the postings list
    
    terms: List[int]
        A list of termIDs to remember the order in which terms and their 
        postings lists were added to index. 
        
        After Python 3.7 we technically no longer need it because a Python dict 
        is an OrderedDict, but since it is a relatively new feature, we still
        maintain backward compatibility with a list to keep track of order of 
        insertion. 
    """
    def __init__(self, index_name, postings_encoding=None, directory=''):
        """
        Parameters
        ----------
        index_name (str): Name used to store files related to the index
        postings_encoding: A class implementing static methods for encoding and 
            decoding lists of integers. Default is None, which gets replaced
            with UncompressedPostings
        directory (str): Directory where the index files will be stored
        """

        self.index_file_path = os.path.join(directory, index_name+'.index')
        self.metadata_file_path = os.path.join(directory, index_name+'.dict')

        if postings_encoding is None:
            self.postings_encoding = UncompressedPostings # 如果编码方式为空 则将其设置上面的编码方式
        else:
            self.postings_encoding = postings_encoding
        # 存放该数据的文件目录: 使用的是 index_name.index  和  index_name.dict来命名
        self.directory = directory
        # 存放对应的文档信息
        self.postings_dict = {}
        # 存放对应的term的信息
        self.terms = []         #Need to keep track of the order in which the 
                                #terms were inserted. Would be unnecessary 
                                #from Python 3.7 onwards

    def __enter__(self):
        """Opens the index_file and loads metadata upon entering the context"""
        # Open the index file
        self.index_file = open(self.index_file_path, 'rb+')

        # Load the postings dict and terms from the metadata file
        # 将对应的文件和term拿出来
        with open(self.metadata_file_path, 'rb') as f:
            self.postings_dict, self.terms = pkl.load(f)
            self.term_iter = self.terms.__iter__()                       

        return self
    
    def __exit__(self, exception_type, exception_value, traceback):
        """Closes the index_file and saves metadata upon exiting the context"""
        # Close the index file
        self.index_file.close()
        
        # Write the postings dict and terms to the metadata file
        # 保存数据
        with open(self.metadata_file_path, 'wb') as f:
            pkl.dump([self.postings_dict, self.terms], f)

因为是在与磁盘上的文件进行交互，所以我们提供了`__enter__`和`__exit__`函数，它使得我们能够像使用python中文件IO一样使用`with`语句。（参考[上下文管理器官方文档](https://docs.python.org/3/library/contextlib.html)）

以下是使用 `InvertedIndexWriter` 上下文管理器的实例:

```python
with InvertedIndexWriter('test', directory='tmp/') as index:
    # Some code here
```

## 索引

> BSBI (i) segments the collection into parts of equal size, (ii) sorts the termID-docID pairs of each part in memory, (iii) stores intermediate sorted results on disk, and (iv) merges all intermediate results into the final index. 

你需要将每一个子目录当做一个块（block），并且在构建索引的过程中每次只能加载一个块（模拟内存不足）。注意到我们是将操作系统意义上的块进行了抽象。你可以认为每个块足够小，能被装载进内存。

在这一部分，我们将阶段性地构造类`BSBIIndex`。函数`index`给出了BSBI算法的框架，而你的工作则是在接下来的部分中实现函数`parse_block`, `invert_write` 和 `merge`。

### 辅助理解

这里需要仔细看，注意其中函数的实现逻辑以及相关的变量：

一、index函数分两步进行:
1. 对每一个块内的文档将建立索引，写入对应名称的索引文件中；
2. 遍历所有的索引文件，对于相同词项的索引进行合并；

二、一些重要变量：
term_id_map和doc_id_map 词典

In [375]:
# Do not make any changes here, they will be overwritten while grading
class BSBIIndex:
    """ 
    Attributes
    ----------
    term_id_map(IdMap): For mapping terms to termIDs
    doc_id_map(IdMap): For mapping relative paths of documents (eg 
        0/3dradiology.stanford.edu_) to docIDs
    data_dir(str): Path to data
    output_dir(str): Path to output index files
    index_name(str): Name assigned to index 
    postings_encoding: Encoding used for storing the postings.
        The default (None) implies UncompressedPostings
    """
    def __init__(self, data_dir, output_dir, index_name = "BSBI", 
                 postings_encoding = None):
        self.term_id_map = IdMap()
        self.doc_id_map = IdMap()
        self.data_dir = data_dir
        self.output_dir = output_dir
        self.index_name = index_name
        self.postings_encoding = postings_encoding

        # Stores names of intermediate indices 存储存储索引的每一个文件的文件名
        self.intermediate_indices = []
        
    def save(self):
        """Dumps doc_id_map and term_id_map into output directory"""
        
        with open(os.path.join(self.output_dir, 'terms.dict'), 'wb') as f:
            pkl.dump(self.term_id_map, f)
        with open(os.path.join(self.output_dir, 'docs.dict'), 'wb') as f:
            pkl.dump(self.doc_id_map, f)
    
    def load(self):
        """Loads doc_id_map and term_id_map from output directory"""
        
        with open(os.path.join(self.output_dir, 'terms.dict'), 'rb') as f:
            self.term_id_map = pkl.load(f)
        with open(os.path.join(self.output_dir, 'docs.dict'), 'rb') as f:
            self.doc_id_map = pkl.load(f)
            
    def index(self):
        """Base indexing code
        
        This function loops through the data directories, 
        calls parse_block to parse the documents : parse_block的作用
        calls invert_write, which inverts each block and writes to a new index
        then saves the id maps and calls merge on the intermediate indices
        """
        for block_dir_relative in sorted(next(os.walk(self.data_dir))[1]):
            td_pairs = self.parse_block(block_dir_relative)
            index_id = 'index_'+block_dir_relative
            self.intermediate_indices.append(index_id)
            with InvertedIndexWriter(index_id, directory=self.output_dir, 
                                     postings_encoding=
                                     self.postings_encoding) as index:
                self.invert_write(td_pairs, index)
                td_pairs = None
        self.save()
        with InvertedIndexWriter(self.index_name, directory=self.output_dir, 
                                 postings_encoding=
                                 self.postings_encoding) as merged_index:
            with contextlib.ExitStack() as stack:
                indices = [stack.enter_context(
                    InvertedIndexIterator(index_id, 
                                          directory=self.output_dir, 
                                          postings_encoding=
                                          self.postings_encoding)) 
                 for index_id in self.intermediate_indices] # for...in语句调用的时迭代器的next方法
                self.merge(indices, merged_index)

### 解析

> The function `parse_block`  parses documents into termID-docID pairs and accumulates the pairs in memory until a block of a fixed size is full. We choose the block size to fit comfortably into memory to permit a fast in-memory sort. 

你需要将每一个子目录当做一个块，`parse_block`接收子目录路径作为参数。同一子目录下所有文件名都是不同的。

_注意 - 我们使用 `BSBIIndex` 继承 `BSBIIndex`, 这只是对一个已存在类添加新内容的简单方法。在这里只是用来切分类的定义（jupyter notebook内教学使用，无特殊含义）。_

In [551]:
class BSBIIndex(BSBIIndex):            
    def parse_block(self, block_dir_relative):
        """Parses a tokenized text file into termID-docID pairs
        本函数的作用: 将已经分好词的文本文件转化为词id和docId对
        Parameters
        ----------
        block_dir_relative : str
            Relative Path to the directory that contains the files for the block
        相对路径
        Returns
        -------
        List[Tuple[Int, Int]]
            Returns all the td_pairs extracted from the block
        返回所有的td_pairs
        Should use self.term_id_map and self.doc_id_map to get termIDs and docIDs.
        需要使用上面提到的term_id_map,doc_id_map得到对应的termID和docID
        These persist across calls to parse_block
        """
        ### Begin your code
        # 0. 初始化变量: 注key值可能重复,我们用列表形式进行存储
        td_pairs=[]
        # 1. 首先确定文件夹路径
        path = os.path.join(self.data_dir,block_dir_relative)
#         print(path)
        # 1.0 判断是否存在文件路径
        if os.path.exists(path) is False:
            raise Exception("The path is not exist")
        # 1.1 判断是否为文件夹:
        if os.path.isdir(path):
            doc_name = block_dir_relative + '\\'
            for doc in os.listdir(path):
                # 2. 建立词id和文档id
                doc_id = self.doc_id_map[doc_name + doc]
                new_path = os.path.join(path,doc)
                for line in open(new_path).readlines():
                    for term in line.strip().split(' '):
#                         print(term, doc)
                        # 3. 产生词项: 文档对
                        td_pairs.append((self.term_id_map[term], doc_id))
        elif os.path.isfile(path):
            with open(path, 'r') as f:
                # 2. 建立词id和文档id
                text = f.read()
                word_list=text.split(' ')
                for word in word_list:
                    word_num = self.term_id_map[word]
                    doc_num = self.doc_id_map[path]
                    # 3. 产生词项: 文档对
                    td_pairs.append([word_num,doc_num])
        else:
            raise Exception("It's a special file(socket,FIFO,device file)")
        # 4. 返回 词项ID-文档ID对
        return td_pairs
        ### End your code

观察函数在测试数据上是否正常运行

In [552]:
with open('toy-data/0/fine.txt', 'r') as f:
    print(f.read())
with open('toy-data/0/hello.txt', 'r') as f:
    print(f.read())

i'm fine , thank you

hi hi
how are you ?



In [553]:
BSBI_instance = BSBIIndex(data_dir=toy_dir, output_dir = 'tmp/', index_name = 'toy')
BSBI_instance.parse_block('0')

[(0, 0),
 (1, 0),
 (2, 0),
 (3, 0),
 (4, 0),
 (5, 1),
 (5, 1),
 (6, 1),
 (7, 1),
 (4, 1),
 (8, 1)]

写一些测试样例来确保`parse_block`方法正常运行（如：相同单词出现时是相同ID）

In [554]:
### Begin your code
BSBI_instance = BSBIIndex(data_dir=toy_dir, output_dir = 'tmp/', index_name = 'toy')
# 1. 验证相同单词是相同ID
# 以toy/1内的文件为例进行验证
td_pairs = BSBI_instance.parse_block('1\\byebye.txt')
assert len(td_pairs) == 1, "The same word has different id"

# 2. 验证相同文章是相同ID
td_pairs = BSBI_instance.parse_block('1\\bye.txt')
td_pairs_np = np.array(td_pairs)
assert len(set(td_pairs_np[:,-1])) == 1, "The same doc has different id"

# 3. 验证找不到文件时抛出异常
try:
    BSBI_instance.parse_block('4')
except Exception as e:
    assert True, "Doesn't raise the exception when the path is not exist."

### End your code

### 倒排表

> The block is then inverted and written to disk. Inversion involves two steps. First, we sort the termID-docID pairs. Next, we collect all termID-docID pairs with the same termID into a postings list, where a posting is simply a docID. The result, an inverted index for the block we have just read, is then written to disk.

在这一部分我们添加函数`invert_write`来实现由termID-docID对构建倒排表。 

但是，我们首先需要实现类`InvertedIndexWriter`。和列表类似，该类提供了append函数，但是倒排表不会存储在内存中而是直接写入到磁盘里。

### 辅助理解

将得到的词项文档对进行构建索引的过程

In [555]:
class InvertedIndexWriter(InvertedIndex):
    """"""
    def __enter__(self):
        self.index_file = open(self.index_file_path, 'wb+')              
        return self

    def append(self, term, postings_list):
        """Appends the term and postings_list to end of the index file.
        将term和postings_list加入index file的文件末尾
        
        This function does three things, 
        1. Encodes the postings_list using self.postings_encoding 将倒排索引解码
        2. Stores metadata in the form of self.terms and self.postings_dict 在term和posting_dict中存储元数据
           Note that self.postings_dict maps termID to a 3 tuple of 
           (start_position_in_index_file, 这个猜测是posting_list在编码文件当中的起始位置
           number_of_postings_in_list, 这个很好确定就是posting_list的长度
           length_in_bytes_of_postings_list) 这个猜测是posting_list在编码文件当中的长度
        3. Appends the bytestream to the index file on disk 输出文件到磁盘上

        Hint: You might find it helpful to read the Python I/O docs
        (https://docs.python.org/3/tutorial/inputoutput.html) for
        information about appending to the end of a file.
        
        Parameters
        ----------
        term:
            term or termID is the unique identifier for the term
        postings_list: List[Int]
            List of docIDs where the term appears
        """
        ### Begin your code
        # term是词项的id postings_list是对应的文章的id列表 
# 问题集合: a.重复的词项如何办呢? 是直接在后面加入还是和之前的合并呢?
#           b.terms需要按顺序存放在磁盘中吗?
# 直接加入文件尾部就好了, 剩下的事情交给invert_write来做

        self.terms.append(term)
        start_pos = self.index_file.tell()
        doc_num = len(postings_list)
        length_of_postings = len(UncompressedPostings.encode(postings_list))
        self.postings_dict[term]=(start_pos,doc_num,length_of_postings)   
#         print(term,postings_list)
        self.index_file.write(UncompressedPostings.encode(postings_list))# 写入文件
        # 这里的异常处理机制?
        ### End your code

尽管还没有实现读取索引的类，我们还是可以用以下测试代码检测我们的实现。

In [556]:
with InvertedIndexWriter('test', directory='tmp/') as index:
    index.append(1, [2, 3, 4])
    index.append(2, [3, 4, 5])
    index.index_file.seek(0)
    assert index.terms == [1,2], "terms sequence incorrect" # 找该index对应的terms列表
    assert index.postings_dict == {1: (0, 3, len(UncompressedPostings.encode([2,3,4]))), # 记录 termId:(起始位置(字节),文档个数,记录长度(字节))
                                   2: (len(UncompressedPostings.encode([2,3,4])), 3, 
                                       len(UncompressedPostings.encode([3,4,5])))}, "postings_dict incorrect" 
    assert UncompressedPostings.decode(index.index_file.read()) == [2, 3, 4, 3, 4, 5], "postings on disk incorrect" # index文件在磁盘的记录

现在我们实现 `invert_write`，它将解析得到的td_pairs转换成倒排表，并使用`InvertedIndexWriter` 类将其写入磁盘。

In [557]:
class BSBIIndex(BSBIIndex):
    def invert_write(self, td_pairs, index):
        """Inverts td_pairs into postings_lists and writes them to the given index
        
        Parameters
        ----------
        td_pairs: List[Tuple[Int, Int]]
            List of termID-docID pairs
        index: InvertedIndexWriter
            Inverted index on disk corresponding to the block       
        """
        ### Begin your code
        # 1. 处理td_pairs中的索引为倒排索引    
        # 1.1 需要将td_pairs分为term和postings_list  
        term_id = -1
        postings_list = []
        for pair in sorted(set(td_pairs)):
            # 1,2 需要将keys值进行排序
            if pair[0] != term_id:
                if term_id != -1:
                    index.append(term_id, set(postings_list))
                term_id = pair[0]
                postings_list = []
            postings_list.append(pair[1])
        index.append(term_id, set(postings_list))
        ### End your code

In [558]:
with InvertedIndexWriter('test', directory='tmp/') as index:
    BSBI_instance = BSBIIndex(data_dir=toy_dir, output_dir = 'tmp/', index_name = 'toy')
    BSBI_instance.invert_write(BSBI_instance.parse_block('0'), index)

我们可以在测试数据上读取一个块并观察倒排索引中包含的内容。
仿照`InvertedIndexWriter`部分写一些测试样例。

In [559]:
### Begin your code
# 1. 读取块
block_dir_relative='0'
BSBI_instance = BSBIIndex(data_dir=toy_dir, output_dir = 'tmp/', index_name = 'toy')
td_pairs = BSBI_instance.parse_block(block_dir_relative)
# 2. 建立索引,存入磁盘
index_id = 'index_'+block_dir_relative
with InvertedIndexWriter(index_id, directory=BSBI_instance.output_dir, 
                            postings_encoding=
                            BSBI_instance.postings_encoding) as index:
    BSBI_instance.invert_write(td_pairs, index)
# 3. 判断索引建立是否正确
    index.index_file.seek(0) # 注: 这个不可缺少, 否则会读不出文件的内容
    assert index.terms == [0,1,2,3,4,5,6,7,8], "terms sequence incorrect" # 找该index对应的terms列表
    assert index.postings_dict == {0: (0, 1, len(UncompressedPostings.encode([0]))),
                               1: (len(UncompressedPostings.encode([0])), 1, len(UncompressedPostings.encode([0]))),
                               2: (len(UncompressedPostings.encode([0,0])), 1, len(UncompressedPostings.encode([0]))),
                               3: (len(UncompressedPostings.encode([0,0,0])), 1, len(UncompressedPostings.encode([0]))),
                               4: (len(UncompressedPostings.encode([0,0,0,0])), 2, len(UncompressedPostings.encode([0,1]))),
                               5: (len(UncompressedPostings.encode([0,0,0,0,0,1])), 1, len(UncompressedPostings.encode([1]))),
                               6: (len(UncompressedPostings.encode([0,0,0,0,0,1,1])), 1, len(UncompressedPostings.encode([1]))),
                               7: (len(UncompressedPostings.encode([0,0,0,0,0,1,1,1])), 1, len(UncompressedPostings.encode([1]))),
                               8: (len(UncompressedPostings.encode([0,0,0,0,0,1,1,1,1])), 1, len(UncompressedPostings.encode([1])))}, "postings_dict incorrect" 
    assert UncompressedPostings.decode(index.index_file.read()) == [0,0,0,0,0,1,1,1,1,1], "postings on disk incorrect" # index文件在磁盘的记录

### End your code

### 合并
> The algorithm simultaneously merges the ten blocks into one large merged index. To do the merging, we open all block files simultaneously, and maintain small read buffers for the ten blocks we are reading and a write buffer for the final merged index we are writing. 

Python中的迭代模型非常自然地符合我们维护一个小的读缓存的要求。我们可以迭代地从磁盘上每次读取文件的一个倒排列表。我们通过构建`InvertedIndex`的子类`InvertedIndexIterator`来完成这个迭代任务。

In [560]:
class InvertedIndexIterator(InvertedIndex):
    """"""
    def __enter__(self):
        """Adds an initialization_hook to the __enter__ function of super class
        """
        super().__enter__()
        self._initialization_hook()
        return self

    def _initialization_hook(self):
        """Use this function to initialize the iterator初始化迭代器的方法
        """
        ### Begin your code
        # 需要一个index_file来读文件
        # 定义初始的迭代器为最开始的文件
        # 1. 初始化文件
#         self.index_file = open(self.index_file_path, 'rb') # 注: 这里打开二进制文件必须以"rb"的形式打开,不然读不出数据内容
        # 2 读取term和postings_dict中的信息找到对应的postings_list,为给next方法做铺垫,我们设置一个下标的变量
        self.indexf = -1 #定义一个新的变量用于找到对应下标的term
#         self.index_file.seek(0)
        
        ### End your code

    def __iter__(self): 
        return self
    
    def __next__(self):
        """Returns the next (term, postings_list) pair in the index.
        返回下一个term,postings_list对
        Note: This function should only read a small amount of data from the 
        index file. In particular, you should not try to maintain the full 
        index file in memory.只需要在index file中读取一小段数据即可
        """
        ### Begin your code
        # 注: 每次读取文件的一个倒排列表 ->意味着每次读取的是其中的一个term和其对应的postings_list
        # 返回的是下一组的迭代器?不是, 返回(term, postings_list)
        # 1. 定位term在terms中的位置,返回新的term和posting_list
        if self.indexf < (len(self.terms)-1):
            self.indexf += 1
            term = self.terms[self.indexf]
            posting = self.postings_dict[term]
            start_pos = posting[0]#起始字节的位置
            length_posting = posting[2]#总的列表长度的位置
            # 2. 读取对应的index_file文件
            self.index_file.seek(start_pos)
            postings_list=UncompressedPostings.decode(self.index_file.read(length_posting)) 
            return (term,postings_list)
        else:
            raise StopIteration
        
        ### End your code

    def delete_from_disk(self):
        """Marks the index for deletion upon exit. Useful for temporary indices
        """
        self.delete_upon_exit = True

    def __exit__(self, exception_type, exception_value, traceback):
        """Delete the index file upon exiting the context along with the
        functions of the super class __exit__ function"""
        self.index_file.close()
        if hasattr(self, 'delete_upon_exit') and self.delete_upon_exit:
            os.remove(self.index_file_path)
            os.remove(self.metadata_file_path)
        else:
            with open(self.metadata_file_path, 'wb') as f:
                pkl.dump([self.postings_dict, self.terms], f)

为了测试以上代码，我们先用`InvertedIndexWriter` 创建索引，然后再迭代遍历。写一些小的测试样例观察它是否正常运行。至少你得写一个测试，手工构建一个小的索引，用`InvertedIndexWriter`将他们写入磁盘，然后用`InvertedIndexIterator`迭代遍历倒排索引。

In [561]:
### Begin your code
# 基本思路是创建索引, 再用迭代器进行迭代遍历, 观察其值是否正确
# 1. 读取块
block_dir_relative='0'
BSBI_instance = BSBIIndex(data_dir=toy_dir, output_dir = 'tmp/', index_name = 'toy')
td_pairs = BSBI_instance.parse_block(block_dir_relative)
# 2. 建立索引,存入磁盘
index_id = 'index_'+block_dir_relative
with InvertedIndexWriter(index_id, directory=BSBI_instance.output_dir, 
                            postings_encoding=
                            BSBI_instance.postings_encoding) as index:
    BSBI_instance.invert_write(td_pairs, index)
# 3. 使用迭代器遍历文件查看索引建立是否正确
with InvertedIndexIterator(index_id, directory=BSBI_instance.output_dir, postings_encoding=BSBI_instance.postings_encoding) as file_iter:
    for i in file_iter:
        print(i)
# 没有问题,可以正常输出
### End your code

(0, [0])
(1, [0])
(2, [0])
(3, [0])
(4, [0, 1])
(5, [1])
(6, [1])
(7, [1])
(8, [1])


> During merging, in each iteration, we select the lowest termID that has not been processed yet using a priority queue or a similar data structure. All postings lists for this termID are read and merged, and the merged list is written back to disk. Each read buffer is refilled from its file when necessary.

我们将使用`InvertedIndexIterator`来完成读取的部分，用`InvertedIndexWriter`来将合并后的索引写入磁盘。

注意到我们之前一直使用`with` 语句来打开倒排索引文件，但是现在需要程序化地完成这项工作。可以看到我们在基类`BSBIIndex`的`index`函数中使用了`contextlib`([参考文档](https://docs.python.org/3/library/contextlib.html#contextlib.ExitStack))
你的任务是合并*打开的*`InvertedIndexIterator`对象（倒排列表），并且通过一个`InvertedIndexWriter`对象每次写入一个文档列表。

既然我们知道文档列表已经排过序了，那么我们可以在线性时间内对它们进行合并排序。事实上`heapq`([参考文档](https://docs.python.org/3.0/library/heapq.html)) 是Python中一个实现了堆排序算法的标准模板。另外它还包含一个实用的函数`heapq.merge`，可以将多个已排好序的输入（可迭代）合并成一个有序的输出并返回它的迭代器。它不仅可以用来合并倒排列表，也可以合并倒排索引词典。

为了让你快速上手`heapq.merge`函数，我们提供了一个简单的使用样例。给出两个以动物和树平均寿命排序的列表，我们希望合并它们。

In [562]:
import heapq
animal_lifespans = [('Giraffe', 28), 
                   ('Rhinoceros', 40), 
                   ('Indian Elephant', 70), 
                   ('Golden Eagle', 80), 
                   ('Box turtle', 123)]

tree_lifespans = [('Gray Birch', 50), 
                  ('Black Willow', 70), 
                  ('Basswood', 100),
                  ('Bald Cypress', 600)]

lifespan_lists = [animal_lifespans, tree_lifespans]

for merged_item in heapq.merge(*lifespan_lists, key=lambda x: x[1]):
    print(merged_item)

('Giraffe', 28)
('Rhinoceros', 40)
('Gray Birch', 50)
('Indian Elephant', 70)
('Black Willow', 70)
('Golden Eagle', 80)
('Basswood', 100)
('Box turtle', 123)
('Bald Cypress', 600)


注意使用`*`将`lifespan_lists`解包作为参数，并且使用`lambda`函数来给出进行排序的key。如果你对它们不熟悉可以参考文档([unpacking lists](https://docs.python.org/3/tutorial/controlflow.html#unpacking-argument-lists), [lambda expressions](https://docs.python.org/3/tutorial/controlflow.html#lambda-expressions))。

In [563]:
import heapq
class BSBIIndex(BSBIIndex):
    def merge(self, indices, merged_index):
        """Merges multiple inverted indices into a single index
        
        Parameters
        ----------
        indices: List[InvertedIndexIterator] 待排序的迭代器序列,每一个都代表一个block中的倒排索引
            A list of InvertedIndexIterator objects, each representing an
            iterable inverted index for a block
        merged_index: InvertedIndexWriter 将合并后的倒排索引序列写入到merged_index当中
            An instance of InvertedIndexWriter object into which each merged 
            postings list is written out one at a time
        """
        ### Begin your code
        # 1. 获取每一个term得到的postings_list
        # 注: 其中每一个term是存储在内存中的,每一个term对应的编号并不相同
        # 1.1 需要遍历每一个迭代器中的term, 然后将其添加进去, term遍历的顺序只需要依次进行增加即可
        # 注: 需要解决的问题是如何将迭代器中的一个一个term找出来, 解决方案是对迭代器进行一个排序, 直到扫描到第一个不是对应的term序号的单词为止
        term = -1
        all_postings_list = []
        for pair in heapq.merge(*indices, key=lambda x: x[0]):
            #对迭代器产生的term对进行排序
            if pair[0] != term:
                # 2 在每次得到的下一个term不为当前term时进行写回
                if term != -1:
                    merged_index.append(term, sorted(set(all_postings_list)))
                term = pair[0]
                all_postings_list = []
            all_postings_list += pair[1]
        # 写回最后一个term
        merged_index.append(term, sorted(set(all_postings_list)))
        ### End your code

首先确保它在测试数据上正常运行

In [564]:
BSBI_instance = BSBIIndex(data_dir=toy_dir, output_dir = 'toy_output_dir', )
BSBI_instance.index()

接下来对整个数据集构建索引

In [565]:
BSBI_instance = BSBIIndex(data_dir='pa1-data', output_dir = 'output_dir', )
BSBI_instance.index()

如果你在合并阶段出现了错误，可以利用以下代码来debug。

In [566]:
BSBI_instance = BSBIIndex(data_dir='pa1-data', output_dir = 'output_dir', )
BSBI_instance.intermediate_indices = ['index_'+str(i) for i in range(10)]
with InvertedIndexWriter(BSBI_instance.index_name, directory=BSBI_instance.output_dir, postings_encoding=BSBI_instance.postings_encoding) as merged_index:
    with contextlib.ExitStack() as stack:
        indices = [stack.enter_context(InvertedIndexIterator(index_id, directory=BSBI_instance.output_dir, postings_encoding=BSBI_instance.postings_encoding)) for index_id in BSBI_instance.intermediate_indices]
        BSBI_instance.merge(indices, merged_index)

## 布尔联合检索 (10%)

我们将实现BSBIIndex中的`retrieve`函数，对于给定的包含由空格分隔tokens的字符串查询，返回包含查询中所有tokens的文档列表。但是我们并不能迭代遍历整个索引或者将整个索引加载到内存来寻找相应的terms（索引太大）。

首先我们要实现`InvertedIndex`的子类`InvertedIndexMapper`，它能够找到对应terms在索引文件中位置并取出它的倒排记录表。

In [567]:
class InvertedIndexMapper(InvertedIndex):
    def __getitem__(self, key):
        return self._get_postings_list(key)
    
    def _get_postings_list(self, term):
        """Gets a postings list (of docIds) for `term`.
        
        This function should not iterate through the index file.
        I.e., it should only have to read the bytes from the index file
        corresponding to the postings list for the requested term.
        """
        ### Begin your code
        # 1. 找到term在索引文件中的位置并取出它的倒排记录表
        posting = self.postings_dict[term]
        start_pos = posting[0]#起始字节的位置
        length_posting = posting[2]#总的列表长度的位置
        # 2. 读取对应的index_file文件
        self.index_file.seek(start_pos)
        postings_list=UncompressedPostings.decode(self.index_file.read(length_posting)) 
        return postings_list
        ### End your code

写一些测试样例检测`_get_postings_list`的实现

In [568]:
### Begin your code
# 打开索引所在位置 查询
with InvertedIndexMapper('BSBI', directory='toy_output_dir/') as index_mapper:
    for i in range(10):
        print(i, index_mapper[i]) 
        # 注: 查询时调用的方法index_mapper[...]而非直接调用_get方法,会有问题
### End your code

0 [0, 4]
1 [0, 4]
2 [0, 2, 4]
3 [0, 4]
4 [0, 1, 2, 4, 5]
5 [1, 5]
6 [1, 5]
7 [1, 5]
8 [1, 2, 5]
9 [2]


现在我们获得了查询中terms对应的倒排记录表，接着需要求他们的交集。这部分与我们之前作业的merge方法类似，请实现`sorted_intersect`函数，遍历两个有序列表并在线性时间内合并。

In [569]:
def sorted_intersect(list1, list2):
    """Intersects two (ascending) sorted lists and returns the sorted result
    
    Parameters
    ----------
    list1: List[Comparable]
    list2: List[Comparable]
        Sorted lists to be intersected
        
    Returns
    -------
    List[Comparable]
        Sorted intersection        
    """
    ### Begin your code
    i = 0
    j = 0
    list3 = []
    while i < len(list1) and j < len(list2):
        if list1[i] < list2[j]:
            i += 1
        elif list1[i] > list2[j]:
            j += 1
        else:
            list3.append(list1[i])
            i += 1
            j += 1
    return list3
    ### End your code

简单的测试样例

In [570]:
### Begin your code
a = [1,3,5,8,20]
b = [2,3,5,9,11,13,15]
sorted_intersect(a,b)
### End your code

[3, 5]

现在可以利用`sorted_intersect` 和 `InvertedIndexMapper`来实现`retrieve`函数。

In [571]:
# %%tee submission/retrieve.py
class BSBIIndex(BSBIIndex):
    def retrieve(self, query):
        """Retrieves the documents corresponding to the conjunctive query
        
        Parameters
        ----------
        query: str
            Space separated list of query tokens
            
        Result
        ------
        List[str]
            Sorted list of documents which contains each of the query tokens. 
            Should be empty if no documents are found.
        
        Should NOT throw errors for terms not in corpus
        """
        if len(self.term_id_map) == 0 or len(self.doc_id_map) == 0:
            self.load()

        ### Begin your code
        # 分词
        words_list = query.split()
        final_list = []
        flag = False
        with InvertedIndexMapper('BSBI', directory=self.output_dir) as mapper:
            for word in words_list:
                # 注: 需要找到对应word对应的id
                word_id = self.term_id_map[word]
                res_list = mapper[word_id]
                # 合并
                # 对于一开始的情况直接进行合并
                if flag is False:
                    final_list = res_list
                    flag = True
                else: 
                    final_list = sorted_intersect(final_list,res_list)
        # 输出
#         print(final_list)
        doc_list = []
        for doc_id in final_list:
            doc_list.append(self.doc_id_map[doc_id])
        return doc_list
#         for doc_id in final_list:
#             print(open(self.data_dir+'/'+self.doc_id_map[doc_id], 'r').read())
        ### End your code

通过一个简单的查询来测试索引在真实数据集上是否正常工作。

In [590]:
BSBI_instance = BSBIIndex(data_dir='pa1-data', output_dir = 'output_dir', )
BSBI_instance.retrieve('boolean retrieval')

['1\\cs276.stanford.edu_',
 '1\\cs276a.stanford.edu_',
 '3\\infolab.stanford.edu_TR_CS-TN-95-21.html',
 '4\\nlp.stanford.edu_IR-book_',
 '4\\nlp.stanford.edu_IR-book_html_htmledition_an-appraisal-of-probabilistic-models-1.html',
 '4\\nlp.stanford.edu_IR-book_html_htmledition_bayesian-network-approaches-to-ir-1.html',
 '4\\nlp.stanford.edu_IR-book_html_htmledition_boolean-retrieval-2.html',
 '4\\nlp.stanford.edu_IR-book_html_htmledition_computing-scores-in-a-complete-search-system-1.html',
 '4\\nlp.stanford.edu_IR-book_html_htmledition_dictionaries-and-tolerant-retrieval-1.html',
 '4\\nlp.stanford.edu_IR-book_html_htmledition_efficient-scoring-and-ranking-1.html',
 '4\\nlp.stanford.edu_IR-book_html_htmledition_inexact-top-k-document-retrieval-1.html',
 '4\\nlp.stanford.edu_IR-book_html_htmledition_probabilistic-information-retrieval-1.html',
 '4\\nlp.stanford.edu_IR-book_html_htmledition_putting-it-all-together-1.html',
 '4\\nlp.stanford.edu_IR-book_html_htmledition_references-and-furth

你也可以通过读取文件来测试其中的页面是否真的包含了查询的terms

In [573]:
with open("pa1-data/1/cs276.stanford.edu_", 'r') as f:
    print(f.read())

cs276 information retrieval and web search cs 276 ling 286 information retrieval and web search spring 2011 pandu nayak and prabhakar raghavan lecture 3 units tu th 4 15 5 30 gates b03 tas sonali aggarwal ivan cui valentin spitkovsky and sandeep sripada staff e mail cs276 spr1011 staff lists stanford edu lectures are also available online and on television through scpd sitn course description basic and advanced techniques for text based information systems efficient text indexing boolean and vector space retrieval models evaluation and interface issues web search including crawling link based algorithms and web metadata text web clustering classification text mining syllabus additional information staff contact information piazzza we strongly recommend students to post questions to the course page on www piazzza com instead of sending emails this forum enables students to discuss the questions they encounter in lectures or assignments here is a quick introduction video email if you hav

测试dev queries（提前构建好的查询与结果）是否正确

In [574]:
BSBI_instance = BSBIIndex(data_dir='pa1-data', output_dir = 'output_dir', )
for i in range(1, 9):
    with open('dev_queries/query.' + str(i)) as q:
        query = q.read()
        my_results = [os.path.normpath(path) for path in BSBI_instance.retrieve(query)]
        with open('dev_output/' + str(i) + '.out') as o:
            reference_results = [os.path.normpath(x.strip()) for x in o.readlines()]
#             print(reference_results)
#             print(set(my_results)-set(reference_results))
            assert set(my_results) == set(reference_results), "Results DO NOT match for query: "+query.strip()
        print("Results match for query:", query.strip())

Results match for query: we are
Results match for query: stanford class
Results match for query: stanford students
Results match for query: very cool
Results match for query: the
Results match for query: a
Results match for query: the the
Results match for query: stanford computer science


如果出现了错误，可以通过以下代码比较输出与正确结果的差异

In [575]:
set(my_results) - set(reference_results)

set()

In [576]:
set(reference_results) - set(my_results)

set()

### 辅助理解

其中set()为空报错的原因在于:
1. 有重复的项被放进去了
2. 顺序不对
注: 这里可能是读取文件的顺序缘故 不去考虑它

确保你构建自己的查询来测试所有可能的边界情况，例如数据集中没有出现的terms，或者拖慢合并的频繁出现的terms。

# 索引压缩  (30%)

在这部分，你将使用可变长字节编码对索引进行压缩。（gap-encoding可选，对gap进行编码）

下面几个Python运算符可能对你有帮助

In [577]:
# Remainder (modulo) operator %

print("10 % 2 = ", 10 % 2)
print("10 % 3 = ", 10 % 3)

# Integer division in Python 3 is done by using two slash signs

print("10 / 3 = ", 10 / 3)
print("10 // 3 = ", 10 // 3)

10 % 2 =  0
10 % 3 =  1
10 / 3 =  3.3333333333333335
10 // 3 =  3


In [578]:
from bitarray import bitarray
# 使用二进制库
# bytesofnum =  bitarray(bin(5)[2:], endian='big')
# print(bin(1)[2:])
# print(len(bin(5)[2:])-1)
bytesofnum2 = bitarray(8-len(bin(5)[2:]))
bytesofnum2.setall(0)
bytesofnum2.extend(bin(5)[2:])
print(bytesofnum2)
bytesofnum = bytesofnum2
print(bytesofnum)
a = bitarray('10000000', endian='big')
b = bitarray(endian='big')
b.frombytes(bytesofnum.tobytes())
print(bytesofnum.tobytes())
print(b)
print(b.to01())

bitarray('00000101')
bitarray('00000101')
b'\x05'
bitarray('00000101')
00000101


完成下面的`CompressedPostings`类，我们将用它来替换`UncompressedPostings`。关于如何使用gap-encoding和可变长字节编码，你可以参考老师课件或者[Chapter 5](https://nlp.stanford.edu/IR-book/pdf/05comp.pdf)。

In [579]:
class CompressedPostings:
    #If you need any extra helper methods you can add them here 
    # 采用VB编码的格式: 其中每个字节的第一位为延续位, 延续位为1结束读取, 延续位为0继续读取
    # 具体的编码方式: 将原来的编码转变为字节 
    ### Begin your code
    @staticmethod
    def VBEncodeNumber(n):
        # 具体的方式将原来的数变为[]的形式,每一个列表中的值就代表一个字节的表示
        bytesofnum = []
        while True:
            mod_value = n % 128
            bytesofnum.insert(0,mod_value)
            if n < 128:
                break
            n = n // 128
        bytesofnum[len(bytesofnum)-1]+=128
        return bytesofnum
    ### End your code
    
    @staticmethod
    def encode(postings_list):
        """Encodes `postings_list` using gap encoding with variable byte 
        encoding for each gap
        
        Parameters
        ----------
        postings_list: List[int]
            The postings list to be encoded
        
        Returns
        -------
        bytes: 
            Bytes reprsentation of the compressed postings list 
            (as produced by `array.tobytes` function)
        """
        ### Begin your code
        encoded_postings_list = []
        # 1. 将第一个元素放进去
        encoded_postings_list += CompressedPostings.VBEncodeNumber(postings_list[0])
        for i in range(1, len(postings_list)):
            # 2. 计算gap然后编码
            encoded_postings_list += CompressedPostings.VBEncodeNumber(postings_list[i] - postings_list[i-1])
        return array.array('B', encoded_postings_list).tobytes()
        ### End your code
        
    @staticmethod
    def decode(encoded_postings_list):
        """Decodes a byte representation of compressed postings list
        
        Parameters
        ----------
        encoded_postings_list: bytes
            Bytes representation as produced by `CompressedPostings.encode` 
            
        Returns
        -------
        List[int]
            Decoded postings list (each posting is a docIds)
        """
        ### Begin your code
        # 1. 获取得到的字节序列
        decoded_postings_list = array.array('B')
        decoded_postings_list.frombytes(encoded_postings_list)
        
        postings_list = []
        # 2. 获取文章id索引
        length = len(decoded_postings_list)
        n = 0 #提取的文章id: 注意是gap+前一个id的值
        first = 0#当前文章前一个id的值
        flag = False
        for i in range(length):
            if decoded_postings_list[i] < 128:
                n = 128*n + decoded_postings_list[i]
            else:
                n = 128*n + decoded_postings_list[i]-128
                if flag is False:
                    first = n
                    postings_list.append(first)
                    n = 0
                    flag = True
                else:
                    n = n + first
                    postings_list.append(n)
                    first = n
                    n = 0       
        return postings_list
        ### End your code


首先，如果你实现了额外的辅助函数，写一些测试样例

In [580]:
### Begin your code
print(CompressedPostings.VBEncodeNumber(824))
print(0b110, 0b10111000)
print(CompressedPostings.VBEncodeNumber(5))
print(0b10000101)
print(CompressedPostings.VBEncodeNumber(214577))
print(0b1101, 0b1100, 0b10110001)
### End your code

[6, 184]
6 184
[133]
133
[13, 12, 177]
13 12 177


我们实现了一个简单的函数来测试你编码的列表是否被正确解码

In [581]:
def test_encode_decode(l):
    e = CompressedPostings.encode(l)
    d = CompressedPostings.decode(e)
    assert d == l
    print(l, e)

写一些测试样例来确保文档列表的压缩和解压正常运行

In [582]:
### Begin your code
list=[[824,829,215406], [1,2,545,2930],[34,454,3213,23132,2133123]]
for l in list:
    test_encode_decode(l)
### End your code

[824, 829, 215406] b'\x06\xb8\x85\r\x0c\xb1'
[1, 2, 545, 2930] b'\x81\x81\x04\x9f\x12\xd1'
[34, 454, 3213, 23132, 2133123] b'\xa2\x03\xa4\x15\xc7\x01\x1b\xcf\x01\x00d\xa7'


现在让我们创建一个新的输出文件夹并使用`CompressedPostings`来构建压缩索引

In [583]:
try: 
    os.mkdir('output_dir_compressed')
except FileExistsError:
    pass

In [584]:
BSBI_instance_compressed = BSBIIndex(data_dir='pa1-data', output_dir = 'output_dir_compressed', postings_encoding=CompressedPostings)
BSBI_instance_compressed.index()

In [585]:
BSBI_instance_compressed = BSBIIndex(data_dir='pa1-data', output_dir = 'output_dir_compressed', postings_encoding=CompressedPostings)
BSBI_instance_compressed.retrieve('boolean retrieval')

['1\\cs276.stanford.edu_',
 '1\\cs276a.stanford.edu_',
 '3\\infolab.stanford.edu_TR_CS-TN-95-21.html',
 '4\\nlp.stanford.edu_IR-book_',
 '4\\nlp.stanford.edu_IR-book_html_htmledition_an-appraisal-of-probabilistic-models-1.html',
 '4\\nlp.stanford.edu_IR-book_html_htmledition_bayesian-network-approaches-to-ir-1.html',
 '4\\nlp.stanford.edu_IR-book_html_htmledition_boolean-retrieval-2.html',
 '4\\nlp.stanford.edu_IR-book_html_htmledition_computing-scores-in-a-complete-search-system-1.html',
 '4\\nlp.stanford.edu_IR-book_html_htmledition_dictionaries-and-tolerant-retrieval-1.html',
 '4\\nlp.stanford.edu_IR-book_html_htmledition_efficient-scoring-and-ranking-1.html',
 '4\\nlp.stanford.edu_IR-book_html_htmledition_inexact-top-k-document-retrieval-1.html',
 '4\\nlp.stanford.edu_IR-book_html_htmledition_probabilistic-information-retrieval-1.html',
 '4\\nlp.stanford.edu_IR-book_html_htmledition_putting-it-all-together-1.html',
 '4\\nlp.stanford.edu_IR-book_html_htmledition_references-and-furth

像之前一样，用已构造好的查询语句来测试是否正常运行

In [587]:
for i in range(1, 9):
    with open('dev_queries/query.' + str(i)) as q:
        query = q.read()
        my_results = [os.path.normpath(path) for path in BSBI_instance_compressed.retrieve(query)]
        with open('dev_output/' + str(i) + '.out') as o:
            reference_results = [os.path.normpath(x.strip()) for x in o.readlines()]
            assert set(my_results) == set(reference_results), "Results DO NOT match for query: "+query.strip()
        print("Results match for query:", query.strip())

Results match for query: we are
Results match for query: stanford class
Results match for query: stanford students
Results match for query: very cool
Results match for query: the
Results match for query: a
Results match for query: the the
Results match for query: stanford computer science


# 额外的编码方式 (10%)

通过补充`ECCompressedPostings`的`encode` 和 `decode`方法来实现一种额外的索引压缩方式。在我们课上学的就是**gamma-encoding** 。另外如果大家感兴趣的话也可以了解**Delta Encoding** ，[ALGORITHM SIMPLE-9](https://github.com/manning/CompressionAlgorithms#simple-9) 等。

你应该以多字节（而不是bits）来存储倒排记录表，因为索引的长度和位置都存的是字节信息。

In [588]:
# 这里实现的是Gamma编码
from bitarray import bitarray
# 使用二进制库更加方便
class ECCompressedPostings:
    #If you need any extra helper methods you can add them here 
    ### Begin your code
    @staticmethod
    def GammaEncodeNumber(n):
        # 具体的方式先将原来的码变成二进制 数位数 然后编进去就好了
        # 问题在于一整个字节的存储: 这个最后再存
        if n == 0:
            bytesofnum = bitarray(0)
            return bytesofnum
        bytesofnum1= bitarray(len(bin(n)[2:])-1, endian='big')
        bytesofnum1.setall(1)
        bytesofnum1.extend('0')
#         print(bytesofnum1)
        bytesofnum2 =  bitarray(bin(n)[3:], endian='big')
#         print(bytesofnum2)
        bytesofnum1.extend(bytesofnum2)
#         print(bytesofnum1)
        return bytesofnum1
    ### End your code
    
    @staticmethod
    def encode(postings_list):
        """Encodes `postings_list` 
        
        Parameters
        ----------
        postings_list: List[int]
            The postings list to be encoded
        
        Returns
        -------
        bytes: 
            Bytes reprsentation of the compressed postings list 
        """
        ### Begin your code
        # 注意: 这里Gamma编码方式需要注意的一点就是需要注意0的编码, 这里采用了一种措施就是将一开始的文章id+1,这样就避免了这个问题
        # (注意gap不需要变化, 因为gap不可能等于0)
        encoded_postings_list = ECCompressedPostings.GammaEncodeNumber(postings_list[0]+1)
        for i in range(1, len(postings_list)):
            encoded_postings_list.extend(ECCompressedPostings.GammaEncodeNumber(postings_list[i] - postings_list[i-1]))
        if len(encoded_postings_list) // 8 !=0:
            res = 8 - len(encoded_postings_list)%8
            bytesofnum= bitarray(res)
            bytesofnum.setall(1)
            encoded_postings_list.extend(bytesofnum)
        return encoded_postings_list.tobytes()
        ### End your code

        
    @staticmethod
    def decode(encoded_postings_list):
        """Decodes a byte representation of compressed postings list
        
        Parameters
        ----------
        encoded_postings_list: bytes
            Bytes representation as produced by `CompressedPostings.encode` 
            
        Returns
        -------
        List[int]
            Decoded postings list (each posting is a docId)
        """
        ### Begin your code
        decoded_bytes_list = bitarray(endian='big')
        decoded_bytes_list.frombytes(encoded_postings_list)
        decoded_bytes_list = decoded_bytes_list.to01()
#         print(decoded_bytes_list)
        first = 0 
        gap = 0
        length = len(decoded_bytes_list)
        flag = False
        postings_list = []
        i = 0
        while i < length:
            bits = 0
            while i < length and decoded_bytes_list[i]=='1':
                bits+=1
                i+=1
            if i < length:
                i = i + 1 + bits
                gap = int('1' + decoded_bytes_list[i-bits : i], 2)
#                 print(gap)
                if flag is False:
                    # 注意对于第一个元素需要减去1
                    first = gap-1
                    postings_list.append(first)
                    gap = 0
                    flag = True
                else:
                    gap = gap + first
                    first = gap
                    postings_list.append(first)
                    gap = 0
        return postings_list
        ### End your code

In [589]:
# 测试通过
def test_encode_decode2(l):
    e = ECCompressedPostings.encode(l)
    d = ECCompressedPostings.decode(e)
    assert d == l
    print(l, e)
list2=[[824,829,215406], [1,2,545,2930],[34,454,3213,23132,2133123],[0,1,3,6,10,19,32,56,567,1592]]
for l in list2:
    test_encode_decode2(l)

[824, 829, 215406] b'\xff\xa79\xff\xff\xa8\xc6?'
[1, 2, 545, 2930] b'\x8f\xf8?\xff\xc5G'
[34, 454, 3213, 23132, 2133123] b'\xf8\x7f\xeaO\xfeX\xff\xff\x8d\xcf\xff\xff\xf8\x06D\xff'
[0, 1, 3, 6, 10, 19, 32, 56, 567, 1592] b'%\xc7\x1e\xbe\x8f\xf7\xff\xfe\x00\x7f'


# 作业提交

本次作业截止日期为10.8。请大家在截止日期前将代码（包含运行结果，测试内容不作要求），实验报告（可单独撰写，也可整合在jupyter notebook中）一起提交到nkulxb2022@163.com，命名方式为`学号_姓名_hw1`。